<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Reinforcement_Learning_(RL)_with_Deep_Q_Learning_for_Game_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

# Define Q-network
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Initialize environment, network, and optimizer
env = gym.make("CartPole-v1", new_step_api=True)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
q_network = QNetwork(state_dim, action_dim)
optimizer = optim.Adam(q_network.parameters(), lr=0.001)

# Hyperparameters
gamma = 0.99
epsilon = 0.1
episodes = 1000

# Training loop
for episode in range(episodes):
    state = env.reset()
    done = False
    while not done:
        # Epsilon-greedy action selection
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            with torch.no_grad():
                action = torch.argmax(q_network(torch.tensor(state).float())).item()

        # Perform action
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        next_state = next_state['observation'] if 'observation' in next_state else next_state

        # Update Q-network
        target = reward + gamma * torch.max(q_network(torch.tensor(next_state).float()))
        loss = nn.MSELoss()(q_network(torch.tensor(state).float())[action], target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state